In [2]:
import requests
import pandas as pd
import numpy as np

In [3]:
url = 'http://127.0.0.1:25510/list/roots?sec=STOCK'
headers = {'Accept': 'application/json'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    # The request was successful
    json_data = response.json()
    print(json_data)
else:
    # Request failed
    print(f"Request failed with status code {response.status_code}")
    print(response.text)

{'header': {'id': 13, 'latency_ms': 115, 'error_type': 'null', 'error_msg': 'null', 'next_page': 'null', 'format': None}, 'response': ['A', 'AA', 'AAA', 'AAAP', 'AAAU', 'AABA', 'AAC', 'AAC.U', 'AAC.WS', 'AACC', 'AACG', 'AACI', 'AACIU', 'AACIW', 'AACOU', 'AACOW', 'AACQ', 'AACQU', 'AACQW', 'AACT', 'AACT.U', 'AACT.WS', 'AADI', 'AADR', 'AAIC', 'AAIC.PR.B', 'AAIC.PR.C', 'AAIN', 'AAIR', 'AAIT', 'AAL', 'AALCP', 'AAM.PR.A', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPB', 'AAPC', 'AAPD', 'AAPH', 'AAPL', 'AAPU', 'AAPY', 'AAQC', 'AAQC.U', 'AAQC.WS', 'AAT', 'AATC', 'AAU', 'AAVL', 'AAWW', 'AAXJ', 'AAXN', 'AB', 'ABAC', 'ABAT', 'ABAX', 'ABB', 'ABBB', 'ABBV', 'ABC', 'ABCB', 'ABCD', 'ABCL', 'ABCM', 'ABCO', 'ABCW', 'ABDC', 'ABEO', 'ABEOW', 'ABEQ', 'ABEV', 'ABFS', 'ABG', 'ABGB', 'ABGI', 'ABHD', 'ABIL', 'ABILW', 'ABIO', 'ABL', 'ABLLW', 'ABLV', 'ABLVW', 'ABLX', 'ABM', 'ABMD', 'ABNB', 'ABOS', 'ABR', 'ABR.PR.E', 'ABSI', 'ABST', 'ABT', 'ABTL', 'ABTX', 'ABUS', 'ABVA', 'ABVC', 'ABVX', 'ABY', 'AC', 'ACA', 

In [4]:
from enum import Enum

class Req(Enum):
    TRADE = 'trade'
    QUOTE = 'quote'
    OI = 'open_interest'

class Right(Enum):
    CALL = 'C'
    PUT = 'P'

In [19]:
class ThetaDataAPI:
    def __init__(self):
        self.base_url = 'http://127.0.0.1:25510/'

    def _get_req(self, url: str, headers: dict, params: dict=None):
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Request failed with status code {response.status_code}")
            print(response.text)

    def get_roots(self, security_type: str='OPTION'):
        url = f'{self.base_url}list/roots?sec={security_type}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

    def get_expirations(self, root:str):
        url = f'{self.base_url}list/expirations?root={root}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_strikes(self, root: str, exp: str):
        url = f'{self.base_url}list/strikes?root={root}&exp={exp}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_dates(self, root: str, security_type: str="option", exp: str=None, strike_right: tuple[str, Right]=None):
        url = f'{self.base_url}list/dates/{security_type}/quote'
        url_params = {}
        url_params["root"] = root
        if exp is not None:
            url_params["exp"] = exp
        if strike_right is not None:
            url_params["strike"] = strike_right[0]
            url_params["right"] = strike_right[1].value
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers, params=url_params)
    
    """
    NOTE this function has a 750ms overhead 
    """
    def get_contracts(self, date: str, req: Req=Req.TRADE):
        url = f'{self.base_url}list/contracts/option/{req.value}?start_date={date}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
        
    def get_eod_prices(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right):
        url = f'{self.base_url}hist/option/eod?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

    def get_hist_quotes(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str=None):
        url = f'{self.base_url}hist/option/quote?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={0 if ivl is None else ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

    def get_ohlc(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str):
        url = f'{self.base_url}hist/option/ohlc?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_hist_oi(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str):
        url = f'{self.base_url}hist/option/open_interest?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_hist_trades(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right):
        url = f'{self.base_url}hist/option/trade?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_hist_iv(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str):
        url = f'{self.base_url}hist/option/implied_volatility?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_hist_iv_verbose(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str):
        url = f'{self.base_url}hist/option/implied_volatility_verbose?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

In [36]:
client = ThetaDataAPI()
root = "AAPL"
client.get_expirations(root=root)

{'header': {'id': 130,
  'latency_ms': 25,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': None},
 'response': [20120601,
  20120608,
  20120616,
  20120622,
  20120629,
  20120706,
  20120713,
  20120721,
  20120727,
  20120803,
  20120810,
  20120818,
  20120824,
  20120831,
  20120907,
  20120914,
  20120922,
  20120928,
  20121005,
  20121012,
  20121020,
  20121026,
  20121102,
  20121109,
  20121117,
  20121123,
  20121130,
  20121207,
  20121214,
  20121222,
  20121228,
  20130104,
  20130111,
  20130119,
  20130125,
  20130201,
  20130208,
  20130216,
  20130222,
  20130301,
  20130308,
  20130316,
  20130322,
  20130328,
  20130405,
  20130412,
  20130420,
  20130426,
  20130503,
  20130510,
  20130518,
  20130524,
  20130531,
  20130607,
  20130614,
  20130622,
  20130628,
  20130705,
  20130712,
  20130720,
  20130726,
  20130802,
  20130809,
  20130817,
  20130823,
  20130830,
  20130906,
  20130913,
  20130921,
  20130927,
  20131004,
  20

In [48]:
class DerivedClient:
    def __init__(self, thetadata: ThetaDataAPI):
        self.thetadata = thetadata

    def get_chains_over_time(self, root: str, exp: str, ivl: str, right: Right, start_date: str=None, end_date: str=None):
        strikes = self.thetadata.get_strikes(root=root, exp=exp)["response"]
        dates = self.thetadata.get_dates(root=root, exp=exp)["response"]
        start = start_date if start_date is not None else dates[0]
        end = end_date if end_date is not None else dates[len(dates)-1]
        strike_count = len(strikes)
        print(strikes)
        print(dates)
        strike_iv = [[] for _ in range(strike_count)]
        for i in range(strike_count):
            K = strikes[i]
            strike_iv[i] = [d[4] for d in self.thetadata.get_hist_iv(root=root, start_date=start, end_date=end, exp=exp, strike=K, right=right, ivl=ivl)["response"]]
        return np.column_stack(strike_iv)



In [50]:
derived_client = DerivedClient(client)
derived_client.get_chains_over_time(root=root, exp=20230406, ivl=3600000, right=Right.CALL, start_date=20230330, end_date=20230331)

[50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 131000, 132000, 133000, 134000, 135000, 136000, 137000, 138000, 139000, 140000, 141000, 142000, 143000, 144000, 145000, 146000, 147000, 148000, 149000, 150000, 152500, 155000, 157500, 160000, 162500, 165000, 167500, 170000, 172500, 175000, 177500, 180000, 182500, 185000, 187500, 190000, 195000, 200000, 205000, 210000, 215000, 220000, 225000, 230000, 235000, 240000, 245000, 250000]
[20230223, 20230224, 20230227, 20230228, 20230301, 20230302, 20230303, 20230306, 20230307, 20230308, 20230309, 20230310, 20230313, 20230314, 20230315, 20230316, 20230317, 20230320, 20230321, 20230322, 20230323, 20230324, 20230327, 20230328, 20230329, 20230330, 20230331, 20230403, 20230404, 20230405, 20230406]


TypeError: 'int' object is not subscriptable

In [35]:
expiries = client.get_expirations(root=root)["response"]
exp = expiries[len(expiries)-50]
strikes = client.get_strikes(root=root, exp=exp)
dates = client.get_dates(root=root, exp=exp)
dates

{'header': {'id': 129,
  'latency_ms': 74,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': None},
 'response': [20230223,
  20230224,
  20230227,
  20230228,
  20230301,
  20230302,
  20230303,
  20230306,
  20230307,
  20230308,
  20230309,
  20230310,
  20230313,
  20230314,
  20230315,
  20230316,
  20230317,
  20230320,
  20230321,
  20230322,
  20230323,
  20230324,
  20230327,
  20230328,
  20230329,
  20230330,
  20230331,
  20230403,
  20230404,
  20230405,
  20230406]}

In [ ]:
contracts = client.get_contracts(date=20220512)
print(contracts["header"])
print(contracts["response"][:10])

In [27]:
eod = client.get_eod_prices(root=root, start_date=20230301, end_date=20230401, exp=20230406, strike=150000, right=Right.CALL)
eod

{'header': {'id': 118,
  'latency_ms': 35,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['open', 'high', 'low', 'close', 'volume', 'count', 'date']},
 'response': [[3.95, 3.95, 3.13, 3.2, 319, 55, 20230301],
  [2.8, 3.5, 2.59, 3.3, 484, 67, 20230302],
  [4.05, 5.6, 3.75, 5.6, 1079, 217, 20230303],
  [7.33, 9.05, 7.0, 7.3, 752, 135, 20230306],
  [7.22, 7.35, 5.6, 5.9, 294, 65, 20230307],
  [6.6, 6.95, 6.2, 6.5, 274, 53, 20230308],
  [6.8, 7.55, 5.35, 5.4, 197, 52, 20230309],
  [5.05, 5.4, 4.01, 4.26, 430, 135, 20230310],
  [4.4, 6.7, 4.16, 5.3, 463, 136, 20230313],
  [5.86, 6.5, 4.88, 5.76, 1547, 160, 20230314],
  [5.55, 6.7, 4.98, 6.32, 301, 73, 20230315],
  [6.2, 8.5, 6.0, 8.15, 176, 54, 20230316],
  [7.75, 8.71, 7.4, 7.7, 79, 37, 20230317],
  [7.7, 9.45, 7.55, 9.23, 62, 40, 20230320],
  [9.71, 10.48, 8.56, 10.43, 88, 36, 20230321],
  [10.85, 12.44, 9.73, 9.73, 145, 58, 20230322],
  [10.3, 12.1, 9.56, 10.37, 83, 37, 20230323],
  [10.07, 10.9, 9.4, 

In [28]:
quotes = client.get_hist_quotes(root=root, start_date=20230331, end_date=20230331, exp=20230406, strike=150000, right=Right.CALL, ivl=60000)
quotes

{'header': {'id': 119,
  'latency_ms': 36,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['ms_of_day',
   'bid_size',
   'bid_condition',
   'bid',
   'bid_exchange',
   'ask_size',
   'ask_condition',
   'ask',
   'ask_exchange',
   'date']},
 'response': [[34200000, 0, 1, 0.0, 50, 0, 1, 0.0, 50, 20230331],
  [34260000, 90, 11, 12.5, 50, 90, 11, 12.9, 50, 20230331],
  [34320000, 90, 11, 12.5, 50, 90, 11, 12.9, 50, 20230331],
  [34380000, 90, 11, 12.5, 50, 90, 11, 12.8, 50, 20230331],
  [34440000, 90, 11, 12.8, 50, 90, 11, 13.05, 50, 20230331],
  [34500000, 90, 11, 13.05, 50, 90, 11, 13.35, 50, 20230331],
  [34560000, 91, 11, 12.9, 50, 90, 11, 13.15, 50, 20230331],
  [34620000, 90, 11, 12.95, 50, 90, 11, 13.2, 50, 20230331],
  [34680000, 90, 11, 12.9, 50, 92, 11, 13.2, 50, 20230331],
  [34740000, 39, 11, 12.95, 50, 93, 11, 13.15, 50, 20230331],
  [34800000, 39, 11, 12.8, 50, 90, 11, 13.0, 50, 20230331],
  [34860000, 90, 11, 12.95, 50, 39, 11, 13.15, 

In [29]:
ohlc = client.get_ohlc(root=root, start_date=20230331, end_date=20230331, exp=20230406, strike=150000, right=Right.CALL, ivl=900000)
ohlc

{'header': {'id': 120,
  'latency_ms': 31,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['ms_of_day',
   'open',
   'high',
   'low',
   'close',
   'volume',
   'count',
   'date']},
 'response': [[34200000, 0.0, 0.0, 0.0, 0.0, 0, 0, 20230331],
  [35100000, 12.64, 12.64, 12.64, 12.64, 1, 1, 20230331],
  [36000000, 12.78, 12.98, 12.78, 12.85, 4, 4, 20230331],
  [36900000, 12.85, 12.85, 12.5, 12.5, 9, 6, 20230331],
  [37800000, 12.73, 12.76, 12.73, 12.73, 22, 4, 20230331],
  [38700000, 12.95, 13.15, 12.95, 13.08, 104, 3, 20230331],
  [39600000, 13.13, 13.13, 12.95, 12.95, 3, 3, 20230331],
  [40500000, 0.0, 0.0, 0.0, 0.0, 0, 0, 20230331],
  [41400000, 13.05, 13.05, 13.05, 13.05, 10, 1, 20230331],
  [42300000, 0.0, 0.0, 0.0, 0.0, 0, 0, 20230331],
  [43200000, 13.35, 13.35, 13.35, 13.35, 11, 2, 20230331],
  [44100000, 13.41, 13.5, 13.41, 13.49, 9, 4, 20230331],
  [45000000, 13.6, 14.03, 13.6, 14.03, 27, 8, 20230331],
  [45900000, 14.07, 14.09, 13.8, 14.

In [30]:
oi = client.get_hist_oi(root=root, start_date=20230301, end_date=20230401, exp=20230406, strike=150000, right=Right.CALL, ivl=900000)
oi

{'header': {'id': 121,
  'latency_ms': 246,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['open_interest', 'date']},
 'response': [[207, 20230301],
  [325, 20230302],
  [716, 20230303],
  [1426, 20230306],
  [1406, 20230307],
  [1474, 20230308],
  [1549, 20230309],
  [1546, 20230310],
  [1640, 20230313],
  [1646, 20230314],
  [1834, 20230315],
  [1835, 20230316],
  [1779, 20230317],
  [1790, 20230320],
  [1781, 20230321],
  [1789, 20230322],
  [1791, 20230323],
  [1791, 20230324],
  [1785, 20230327],
  [1752, 20230328],
  [1784, 20230329],
  [1868, 20230330],
  [1886, 20230331]]}

In [31]:
trades = client.get_hist_trades(root=root, start_date=20230331, end_date=20230331, exp=20230406, strike=150000, right=Right.CALL)
trades

{'header': {'id': 122,
  'latency_ms': 25,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['ms_of_day', 'sequence', 'size', 'condition', 'price', 'date']},
 'response': [[34300186, -1672342590, 1, -126, 12.77, 20230331],
  [34338300, -1670630014, 1, -126, 12.64, 20230331],
  [35512400, -1614731336, 1, -125, 12.78, 20230331],
  [35816185, -1602262760, 1, -125, 12.98, 20230331],
  [35990760, -1595889146, 1, 18, 12.85, 20230331],
  [35994926, -1595657276, 1, 18, 12.85, 20230331],
  [36165588, -1586743848, 1, 18, 12.85, 20230331],
  [36165591, -1586741746, 2, 18, 12.85, 20230331],
  [36351845, -1579569161, 3, -126, 12.69, 20230331],
  [36522714, -1573314180, 1, -125, 12.77, 20230331],
  [36761354, -1564427328, 1, 18, 12.55, 20230331],
  [36789304, -1563300172, 1, 18, 12.5, 20230331],
  [37035124, -1554145621, 18, -126, 12.73, 20230331],
  [37455803, -1539004679, 2, -126, 12.76, 20230331],
  [37458791, -1538872291, 1, -126, 12.76, 20230331],
  [37522847, -

In [32]:
iv = client.get_hist_iv(root=root, start_date=20230301, end_date=20230331, exp=20230406, strike=150000, right=Right.CALL, ivl=3600000)
iv

{'header': {'id': 123,
  'latency_ms': 14552,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['ms_of_day',
   'bid',
   'bid_implied_vol',
   'midpoint',
   'implied_vol',
   'ask',
   'ask_implied_vol',
   'underlying_price',
   'date']},
 'response': [[36000000,
   3.5,
   0.2478,
   3.9,
   0.2701,
   4.3,
   0.292,
   146.92,
   20230301],
  [39600000, 3.65, 0.2671, 3.7, 0.2699, 3.75, 0.2725, 146.47, 20230301],
  [43200000, 3.65, 0.2672, 3.67, 0.2687, 3.7, 0.2703, 146.45, 20230301],
  [46800000, 3.6, 0.2664, 3.62, 0.2674, 3.65, 0.2691, 146.38, 20230301],
  [50400000, 3.4, 0.267, 3.42, 0.2682, 3.45, 0.27, 145.87, 20230301],
  [54000000, 3.25, 0.2677, 3.27, 0.2693, 3.3, 0.2702, 145.48, 20230301],
  [36000000, 2.74, 0.2669, 2.76, 0.2681, 2.78, 0.2693, 144.34, 20230302],
  [39600000, 2.75, 0.2643, 2.76, 0.2649, 2.78, 0.2661, 144.48, 20230302],
  [43200000, 2.69, 0.2635, 2.71, 0.2647, 2.73, 0.2659, 144.35, 20230302],
  [46800000, 2.71, 0.2626, 2.73, 0.